In [1]:
import pandas as pd
import torch
import numpy as np
from vllm import LLM, SamplingParams
from tqdm.auto import tqdm
import random
import os

2024-11-05 11:05:06.284640: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-05 11:05:06.301778: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-05 11:05:06.306823: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-05 11:05:06.321445: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-05 11:05:09.001256: W tensorflow/compiler/tf2

In [2]:
seed = 42
# seed = 20241027
# seed = 20240617
# seed = 355643

def set_seed(seed = 42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    print('> SEEDING DONE')

set_seed(seed)

> SEEDING DONE


In [3]:
model_id = "Qwen/Qwen2.5-Math-72B-Instruct"
num_gpu = torch.cuda.device_count()

llm = LLM(
    model_id,
    tensor_parallel_size=num_gpu,
    gpu_memory_utilization=0.95,
    trust_remote_code=True,
    dtype=torch.bfloat16,
    enforce_eager=True,
    max_model_len=4096,
    disable_log_stats=True,
    max_num_seqs=256,
)
tokenizer = llm.get_tokenizer()

sampling_params = SamplingParams(
    n=1,  # Number of output sequences to return for each prompt.
    top_p=0.8,  # Float that controls the cumulative probability of the top tokens to consider.
    top_k=20,
    temperature=0.7,  # randomness of the sampling
    repetition_penalty=1.05,  # Float that controls the penalty for repeating the same token.
    seed=seed,  # Seed for reprodicibility
    skip_special_tokens=True,  # Whether to skip special tokens in the output.
    max_tokens=2048,  # Maximum number of tokens to generate per output sequence.
)

INFO 11-05 11:05:28 config.py:887] Defaulting to use mp for distributed inference
WARNING 11-05 11:05:28 config.py:380] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 11-05 11:05:28 llm_engine.py:237] Initializing an LLM engine (vdev) with config: model='Qwen/Qwen2.5-Math-72B-Instruct', speculative_config=None, tokenizer='Qwen/Qwen2.5-Math-72B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=4, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=True, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=Obse

Loading safetensors checkpoint shards:   0% Completed | 0/37 [00:00<?, ?it/s]


(VllmWorkerProcess pid=242894) INFO 11-05 11:06:09 model_runner.py:1071] Loading model weights took 33.9765 GB
(VllmWorkerProcess pid=242898) INFO 11-05 11:06:09 model_runner.py:1071] Loading model weights took 33.9765 GB
INFO 11-05 11:06:09 model_runner.py:1071] Loading model weights took 33.9765 GB
(VllmWorkerProcess pid=242893) INFO 11-05 11:06:09 model_runner.py:1071] Loading model weights took 33.9765 GB
INFO 11-05 11:06:11 distributed_gpu_executor.py:57] # GPU blocks: 30729, # CPU blocks: 3276
INFO 11-05 11:06:11 distributed_gpu_executor.py:61] Maximum concurrency for 4096 tokens per request: 120.04x


In [ ]:
df1 = pd.read_csv('../../../../data/2.synthetic_data_generation/generation1_openai.csv')
df2 = pd.read_csv('../../../../data/2.synthetic_data_generation/generation1_llama.csv')
df3 = pd.read_csv('../../../../data/2.synthetic_data_generation/generation1_qwen.csv')
df = pd.concat([df1, df2, df3], ignore_index=True)
df

# get the wrong answer under misconception

In [5]:
system_prompt = 'You are a student with some math misconceptions. Your task is to answer a math question as a student who doesn’t understand the concepts completely and has some math misconceptions. Your answer should be based on the given misconception. If you cannot get the answer with the misconception, answer "Not enough information". DO NOT give the correct answer! Please reason step by step, and put your final answer within \\boxed{}.'

prompt = "Solve the problem under the constraint(misconception): {misconception}. Question: {question}."

In [6]:
i = 0
row = df.iloc[i]
misconception = row['MisconceptionName']
question = row['QuestionText']

# CoT
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": prompt.format(misconception=misconception, question=question)},
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
print(text)

<|im_start|>system
You are a student with some math misconceptions. Your task is to answer a math question as a student who doesn’t understand the concepts completely and has some math misconceptions. Your answer should be based on the given misconception. If you cannot get the answer with the misconception, answer "Not enough information". DO NOT give the correct answer! Please reason step by step, and put your final answer within \boxed{}.<|im_end|>
<|im_start|>user
Solve the problem under the constraint(misconception): Does not know that angles in a triangle sum to 180 degrees. Question: What is the sum of the three angles in any triangle?.<|im_end|>
<|im_start|>assistant



In [7]:
output = llm.generate(text, sampling_params=sampling_params)
print(output[0].outputs[0].text)

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts: 100%|██████████| 1/1 [00:10<00:00, 10.28s/it, est. speed input: 13.43 toks/s, output: 17.32 toks/s]

Given the constraint that the student does not know that the angles in a triangle sum to 180 degrees, we need to determine how they might approach the problem.

The correct mathematical fact is that the sum of the interior angles in any triangle is always 180 degrees. However, since the student does not know this, they might not be able to provide the correct answer based on their lack of understanding.

If the student does not know this fundamental property of triangles, they might try to use other information or make an educated guess. Without any additional context or specific information about the angles, the student would not have enough information to determine the sum of the angles.

Therefore, under the constraint of not knowing that the angles in a triangle sum to 180 degrees, the student's answer would be:

\[
\boxed{\text{Not enough information}}
\]


In [8]:
all_texts = []
for i in tqdm(range(len(df))):
    row = df.iloc[i]
    misconception = row['MisconceptionName']
    question = row['QuestionText']

    # CoT
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt.format(misconception=misconception, question=question)},
    ]

    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    all_texts.append(text)

  0%|          | 0/22370 [00:00<?, ?it/s]

In [9]:
output = llm.generate(all_texts, sampling_params=sampling_params)

In [10]:
output_texts = [x.outputs[0].text for x in output]
output_texts[0:10]

['To solve the problem under the constraint that the student does not know that the angles in a triangle sum to 180 degrees, we need to approach the problem without using this fact directly. However, it is important to note that the sum of the angles in a triangle is a fundamental property of triangles in Euclidean geometry, and it is always 180 degrees.\n\nSince the student does not know this property, they might try to reason about it using other geometric properties or examples, but they would likely not arrive at the correct answer without this key piece of knowledge. Therefore, the student might give an incorrect answer based on their misunderstanding or lack of knowledge.\n\nHowever, as an AI, I can still provide the correct answer to the problem for reference:\n\nThe sum of the three angles in any triangle is \\(\\boxed{180}\\) degrees.',
 "To solve the problem under the constraint that the student does not know that the angles in a triangle sum to 180 degrees, we need to approa

In [11]:
exp_name = f's000-AnswerText-qwen-{seed}'
df[f'{exp_name}'] = output_texts
df

,MisconceptionId,MisconceptionName,ThirdSubjectName,QuestionText,author,ThirdSubjectId,SecondSubjectId,SecondSubjectName,FirstSubjectId,FirstSubjectName,s000-AnswerText-qwen-42
0,0,Does not know that angles in a triangle sum to...,Angles in Triangles,What is the sum of the three angles in any tri...,llama,439,74.0,Angles,71.0,Geometry and Measure,To solve the problem under the constraint that...
1,0,Does not know that angles in a triangle sum to...,Angles in Triangles,"In a triangle, the first angle is 60 degrees a...",llama,439,74.0,Angles,71.0,Geometry and Measure,To solve the problem under the constraint that...
2,0,Does not know that angles in a triangle sum to...,Angles in Triangles,A triangle has two angles that measure 45 degr...,llama,439,74.0,Angles,71.0,Geometry and Measure,To solve the problem under the constraint that...
3,0,Does not know that angles in a triangle sum to...,Angles in Triangles,Can a triangle have two right angles (90 degre...,llama,439,74.0,Angles,71.0,Geometry and Measure,To determine whether a triangle can have two r...
4,0,Does not know that angles in a triangle sum to...,Angles in Triangles,If the sum of two angles in a triangle is 120 ...,llama,439,74.0,Angles,71.0,Geometry and Measure,To solve the problem under the constraint that...
...,...,...,...,...,...,...,...,...,...,...,...
22365,2053,"When reading integers on a number line, assume...",Adding and Subtracting Negative Numbers,A number line shows the interval from -3 to 7....,qwen,211,42.0,Negative Numbers,32.0,Number,To solve the problem under the given constrain...
22366,2053,"When reading integers on a number line, assume...",Place Value,"On a number line, the distance between 2 and 1...",qwen,202,144.0,Basic Arithmetic,32.0,Number,To solve the problem under the given constrain...
22367,2053,"When reading integers on a number line, assume...",Adding and Subtracting Negative Numbers,A number line from -10 to 10 is divided into 8...,qwen,211,42.0,Negative Numbers,32.0,Number,To solve the problem under the given constrain...
22368,2179,Assumes that an arrow on a number line starts ...,Written Subtraction,"On a number line, if an arrow starts at 5 and ...",qwen,206,144.0,Basic Arithmetic,32.0,Number,To solve the problem under the constraint that...


# solve the correct answer

In [13]:
prompt = "{question}"

In [14]:
all_texts = []
for i in tqdm(range(len(df))):
    row = df.iloc[i]
    misconception = row['MisconceptionName']
    question = row['QuestionText']

    # CoT
    messages = [
        {"role": "system", "content": "Please reason step by step, and put your final answer within \\boxed{}."},
        {"role": "user", "content": prompt.format(question=question)},
    ]

    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    all_texts.append(text)

  0%|          | 0/22370 [00:00<?, ?it/s]

In [15]:
output = llm.generate(all_texts, sampling_params=sampling_params)

Processed prompts: 100%|██████████| 22370/22370 [55:43<00:00,  6.69it/s, est. speed input: 370.37 toks/s, output: 1789.47 toks/s]  


In [20]:
output_texts = [x.outputs[0].text for x in output]
output_texts[0:10]

["To determine the sum of the three angles in any triangle, we can use the properties of triangles and parallel lines. Here is a step-by-step explanation:\n\n1. **Consider a Triangle**: Let's take a triangle with angles \\(A\\), \\(B\\), and \\(C\\).\n\n2. **Draw a Parallel Line**: Draw a line through the vertex of angle \\(A\\) that is parallel to the side opposite to angle \\(A\\). This line will create two new angles at the vertex of angle \\(A\\) that are equal to angles \\(B\\) and \\(C\\) because they are corresponding angles.\n\n3. **Identify the New Angles**: The angle on the left side of the drawn line will be equal to angle \\(B\\) and the angle on the right side of the drawn line will be equal to angle \\(C\\).\n\n4. **Sum of Angles on a Straight Line**: Since these new angles and angle \\(A\\) form a straight line, their sum must be 180 degrees. Therefore, we have:\n   \\[\n   A + B + C = 180^\\circ\n   \\]\n\n5. **Conclusion**: The sum of the three angles in any triangle i

In [21]:
exp_name = f's000-CorrectAnswerText-qwen-{seed}'
df[f'{exp_name}'] = output_texts
df

,MisconceptionId,MisconceptionName,ThirdSubjectName,QuestionText,author,ThirdSubjectId,SecondSubjectId,SecondSubjectName,FirstSubjectId,FirstSubjectName,s000-AnswerText-qwen-42,s000-CorrectAnswerText-qwen-42
0,0,Does not know that angles in a triangle sum to...,Angles in Triangles,What is the sum of the three angles in any tri...,llama,439,74.0,Angles,71.0,Geometry and Measure,To solve the problem under the constraint that...,To determine the sum of the three angles in an...
1,0,Does not know that angles in a triangle sum to...,Angles in Triangles,"In a triangle, the first angle is 60 degrees a...",llama,439,74.0,Angles,71.0,Geometry and Measure,To solve the problem under the constraint that...,To determine the measure of the third angle in...
2,0,Does not know that angles in a triangle sum to...,Angles in Triangles,A triangle has two angles that measure 45 degr...,llama,439,74.0,Angles,71.0,Geometry and Measure,To solve the problem under the constraint that...,To determine the measure of the remaining angl...
3,0,Does not know that angles in a triangle sum to...,Angles in Triangles,Can a triangle have two right angles (90 degre...,llama,439,74.0,Angles,71.0,Geometry and Measure,To determine whether a triangle can have two r...,To determine whether a triangle can have two r...
4,0,Does not know that angles in a triangle sum to...,Angles in Triangles,If the sum of two angles in a triangle is 120 ...,llama,439,74.0,Angles,71.0,Geometry and Measure,To solve the problem under the constraint that...,To determine the measure of the third angle in...
...,...,...,...,...,...,...,...,...,...,...,...,...
22365,2053,"When reading integers on a number line, assume...",Adding and Subtracting Negative Numbers,A number line shows the interval from -3 to 7....,qwen,211,42.0,Negative Numbers,32.0,Number,To solve the problem under the given constrain...,To determine the number that the second dash r...
22366,2053,"When reading integers on a number line, assume...",Place Value,"On a number line, the distance between 2 and 1...",qwen,202,144.0,Basic Arithmetic,32.0,Number,To solve the problem under the given constrain...,To determine the number at the first dash afte...
22367,2053,"When reading integers on a number line, assume...",Adding and Subtracting Negative Numbers,A number line from -10 to 10 is divided into 8...,qwen,211,42.0,Negative Numbers,32.0,Number,To solve the problem under the given constrain...,To determine the number that the third dash fr...
22368,2179,Assumes that an arrow on a number line starts ...,Written Subtraction,"On a number line, if an arrow starts at 5 and ...",qwen,206,144.0,Basic Arithmetic,32.0,Number,To solve the problem under the constraint that...,To determine the length of the arrow on a numb...


In [ ]:
import re


correct = []
ct = 0
for i,txt in enumerate(df[f's000-CorrectAnswerText-qwen-{seed}']):
    matches = re.findall(r"\\boxed\{(.*)\}", txt)
    try:
        an = matches[0]
        an = re.sub(r"\\text\{(.*?)\}", r"\1", an)
    except:
        an = 'unknown'
            
    correct.append(an)


In [96]:
df[f'CorrectAnswerText-qwen-seed{seed}'] = correct
df = df[df['CorrectAnswerText-qwen-seed42']!='unknown'].reset_index(drop=True)


In [ ]:
import re

wrong = []
ct = 0
for i,txt in enumerate(df[f's000-AnswerText-qwen-{seed}']):
    matches = re.findall(r"\\boxed\{(.*)\}", txt)

    try:
        an = matches[0]
        an = re.sub(r"\\text\{(.*?)\}", r"\1", an)
    except:
        an = 'unknown'
            
    wrong.append(an)


In [ ]:
df[f'AnswerText-qwen-seed{seed}'] = wrong
df[df[f'AnswerText-qwen-seed{seed}']!='unknown'].reset_index(drop=True)

In [ ]:
df = df[df[f'AnswerText-qwen-seed{seed}']!=df[f'CorrectAnswerText-qwen-seed{seed}']].reset_index(drop=True)
df

In [ ]:
df['CorrectAnswerText'] = df[f'CorrectAnswerText-qwen-seed{seed}']
df['AnswerText'] = df[f'AnswerText-qwen-seed{seed}']
df['CorrectAnswer'] = 'D'
df['Answer'] = 'A'
question_ids = []
for i in range(len(df)):
    question_ids.append(f'generation1-{i}')
df['QuestionId_Answer'] = df.apply(lambda x: f"{x['QuestionId']}_{x['Answer']}", axis=1)
df['SubjectId'] = '-1'

In [ ]:
df.to_csv(f'../../../../data/2.synthetic_data_generation/generation1_all_render.csv', index=False)